# Temporal Vectors

## Abstract

In this notebook, I will conceptualize Temporal Vectors. Temporal vectors are a way to encode the temporal information of a time series. They are used to encode movement pattern of a sequence. This information is very useful in time series forecasting tasks. I will show how to use temporal vectors in a simple model to forecast the number of passengers in a time series.
